<a href="https://colab.research.google.com/github/Quartz/aistudio-workshops/blob/master/jj-searching-documents-with-use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Search with Sentence Encoder

Quartz Nicar etc.

https://github.com/Quartz/aistudio-searching-data-dumps-with-use

# Getting Started

This section sets up the environment for access to the Multilingual Universal Sentence Encoder Module and also prepares a set of English sentences and their translations. In the following sections, the multilingual module will be used to compute similarity *across languages*.

**IMPORTANT**Note: Pleaseelect "**Python 3**" _and_ "**GPU**" in the ***Runtime->Change Runtime type*** dropdown menu above _before_ running this notebook for faster execution.

In [2]:
#@title Setup Environment
#latest Tensorflow that supports sentencepiece is 1.14
!pip uninstall --quiet --yes tensorflow
!pip install --quiet tensorflow-gpu==1.14
!pip install --quiet tensorflow==1.14
!pip install --quiet tensorflow-hub
!pip install --quiet bokeh
!pip install --quiet tf-sentencepiece
!pip install --quiet annoy
!pip install --quiet tqdm
!pip install --quiet w3lib
!pip install --quiet syntok


     |████████████████████████████████| 377.0MB 42kB/s 
     |████████████████████████████████| 491kB 39.5MB/s 
     |████████████████████████████████| 3.2MB 24.9MB/s 
     |████████████████████████████████| 109.2MB 29kB/s 
     |████████████████████████████████| 1.4MB 2.8MB/s 
     |████████████████████████████████| 645kB 2.7MB/s 


In [3]:
#@title Setup common imports and functions
%tensorflow_version 1.x
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tf_sentencepiece  # Not used directly but needed to import TF ops.
import sklearn.metrics.pairwise

from tqdm import tqdm
from tqdm import trange
from annoy import AnnoyIndex

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

This is additional boilerplate code where we import the pre-trained ML model we will use to encode text throughout this notebook.

In [4]:
# this version of the Universal Sentence Encoder only "speaks" English
# but there's another version you can switch in that supports 16 different languages!
module_url = 'https://tfhub.dev/google/universal-sentence-encoder/2'

# boilerplate, getting started with Tensorflow.
# (how to use Tensorflow is way outside the scope of this class)
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  multiling_embed = hub.Module(module_url)
  embedded_text = multiling_embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

session = tf.Session(graph=g)
session.run(init_op)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:
# let's get our data!
# it's a JSONL file, which is a file with one page, as its own JSON document, per line.
!wget -nc -O nyc_docs.jsonl https://raw.githubusercontent.com/Quartz/aistudio-searching-data-dumps-with-use/master/nyc_docs.jsonl

--2020-01-29 22:58:06--  https://raw.githubusercontent.com/Quartz/aistudio-searching-data-dumps-with-use/master/nyc_docs.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5893016 (5.6M) [text/plain]
Saving to: ‘nyc_docs.jsonl’

nyc_docs.jsonl      100%[===================>]   5.62M  20.8MB/s    in 0.3s    

2020-01-29 22:58:07 (20.8 MB/s) - ‘nyc_docs.jsonl’ saved [5893016/5893016]



## Chopping each page into a list of sentences

We have to do this because pages and paragraphs often cover multiple topics, which might confuse the model. And, Universal Sentence Encoder is built to encode sentences... and so it ignores anything after the 128th word in its input.

The code below cuts the text into sentences, but groups any two consecutive sentences under 15 words long together.

In [7]:
# takes about 10 seconds

import json
from bs4 import BeautifulSoup
from functools import reduce
from w3lib.html import remove_tags

import syntok.segmenter as segmenter

total_docs = 4251 # get this with `wc` (only used for progress bar)

total_short_paragraphs = 0
MAX_SENT_LEN = 100

def sentenceify(text):
    return [sl for l in [[''.join([t.spacing + t.value for t in s]) for s in p if len(s) < MAX_SENT_LEN] for p in segmenter.analyze(text)] for sl in l if any(map(lambda x: x.isalpha(), sl))]


def clean_html(html):
    if "<" in html and ">" in html:
        try:
            soup = BeautifulSoup(html, features="html.parser")
            plist = soup.find('plist')
            if plist:
                plist.decompose() # remove plists because ugh
            text = soup.getText()
        except:
            text = remove_tags(html)
        return '. '.join(text.split("\r\n\r\n\r\n"))
    else:
        return '. '.join(html.split("\r\n\r\n\r\n"))

# if this sentence is short, then group it with other short sentences (so you get groups of continuous short sentences, broken up by one-element groups of longer sentences)
def short_sentence_grouper_bean_factory(target_sentence_length): # in chars
    def group_short_sentences(list_of_lists_of_sentences, next_sentence):
        if not list_of_lists_of_sentences:
            return [[next_sentence]]
        if len(next_sentence) < target_sentence_length:
           list_of_lists_of_sentences[-1].append(next_sentence)
        else:
            list_of_lists_of_sentences.append([next_sentence])
            list_of_lists_of_sentences.append([])
        return list_of_lists_of_sentences
    return group_short_sentences


def overlap(document_tokens, target_length):
    """ pseudo-paginate a document by creating lists of tokens of length `target-length` that overlap at 50%
    return a list of `target_length`-length lists of tokens, overlapping by 50% representing all the tokens in the document 
    """

    overlapped = []
    cursor = 0
    while len(' '.join(document_tokens[cursor:]).split()) >= target_length:
      overlapped.append(document_tokens[cursor:cursor+target_length])
      cursor += target_length // 2
    return overlapped


def sentences_to_short_paragraphs(group_of_sentences, target_length, min_shingle_length=10):
    """ outputting overlapping groups of shorter sentences 
    
        group_of_sentences = list of strings, where each string is a sentences
        target_length = max length IN WORDS of output sentennces
        min_shingle_length = don't have sentences that differ just in the inclusion of a sentence of this size
    """
    if len(group_of_sentences) == 1:
        return [' '.join(group_of_sentences[0].split())]
    sentences_as_words = [sent.split() for sent in group_of_sentences]
    sentences_as_words = [sentence for sentence in sentences_as_words if [len(word) for word in sentence].count(1) < (len(sentence) * 0.5) ]
    paragraphs = []
    for i, sentence in enumerate(sentences_as_words[:-1]):
        if i > 0 and len(sentence) < min_shingle_length  and len(sentences_as_words[i-1]) < min_shingle_length and i % 2 == 0:
            continue # skip really short sentences if the previous one is also really short (but not so often that we lose anything )
        buff = list(sentence) # just making a copy.
        for subsequent_sentence in sentences_as_words[i+1:]:
            if len(buff) + len(subsequent_sentence) <= target_length:
                buff += subsequent_sentence
            else:
                break
        paragraphs.append(buff)
    return [' '.join(graf) for graf in paragraphs]


def to_short_paragraphs(text, paragraph_len=15, min_sentence_len=8): # paragraph_len in words, min_sentence_len in chars
    sentences = sentenceify( clean_html(text) )
    grouped_sentences = reduce(short_sentence_grouper_bean_factory(150) , sentences, [])
    return [sl for l in [sentences_to_short_paragraphs(group, paragraph_len) for group in grouped_sentences if len(group) >= 2 or (len(group) > 0 and len(group[0]) > min_sentence_len)] for sl in l if sl]

paragraph_target_length = 15

with open(f"nyc_docs-sentences{paragraph_target_length}.json", 'w') as writer: 
    with open('nyc_docs.jsonl', 'r') as reader:
        for i, line_json in tqdm(enumerate(reader), total=total_docs):
            line = json.loads(line_json)
            text = line["_source"]["content"][:1000000]
            for j, page in enumerate(to_short_paragraphs(text, paragraph_target_length)):
                total_short_paragraphs += 1
                writer.write(json.dumps({
                    "text": page, 
                    "_id": line["_id"], 
                    "chonk": j,
                    # "routing": line.get("_routing", None),
                    # "path": line["_source"]["path"]
                    }) + "\n")
print(f"total paragraphs: {total_short_paragraphs}")


100%|██████████| 4251/4251 [00:14<00:00, 287.79it/s]

total paragraphs: 37281


# Creating a Multilingual Semantic-Similarity Search Engine

## Using a pre-trained model to transform sentences into vectors

We compute embeddings in _batches_ so that they fit in the GPU's RAM.

In [8]:
# Takes about 12 seconds
vector_index_chunk = AnnoyIndex(512, 'angular')  # Length of item vector that will be indexed

batch_size = 256
docs = {}

doc_counter = 0
with tqdm(total=37281) as pbar:
  for j, batch in enumerate(pd.read_json('nyc_docs-sentences15.json', lines=True, chunksize=batch_size)):
    batch_vecs = session.run(embedded_text, feed_dict={text_input: batch["text"]})
    # sentences.extend(batch["text"])
    pbar.update(len(batch))
    doc_idxs = list(range(doc_counter, doc_counter + batch_size))
    for vec, page_num, doc in zip(batch_vecs, doc_idxs, batch.iterrows()):
      vector_index_chunk.add_item(page_num, vec)
      docs[page_num] = doc[1]["_id"]
    doc_counter += batch_size
    
    

100%|██████████| 37281/37281 [00:16<00:00, 2301.46it/s]


## Building an index of semantic vectors

We use the [Annoy](https://github.com/spotify/annoy) library---to efficiently look up results from the corpus.

In [9]:
vector_index_chunk.build(10) # 10 trees
vector_index_chunk.save('nyc_docs_annoy_small.bin') # you could save this and skip the step above, if you'd like



True

What's indexed in Annoy is a meaningless set of 512 numbers for each sentence. Computers can sort of understand this, but humans can't. So we load up into memory the list of all the sentences, so we can print those as the result.

This demo uses a fairly small (5mb) set of documents. If you were using this in "real life" you'd probably want to use a database to hold onto these -- they'd be too big to hold in memory.

In [0]:
doc_texts = pd.read_json('nyc_docs-sentences15.json', lines=True);

## Verify that the semantic-similarity search engine works

Let's search for some stuff!

*   Try a few different sample sentences
*   Try changing the number of returned results (they are returned in order of similarity)

Once you've tried it out a bit, click the menu button to the left, and click Form -> Show Code to see what this is doing under the hood.


In [20]:
sample_query = 'Subway times are getting worse'  #@param ["Global warming", "Researchers made a surprising new discovery last week.", "The stock market fell four points.", "Lawmakers will vote on the proposal tomorrow."] {allow-input: true}
num_results = 10  #@param {type:"slider", min:0, max:50, step:1}

query_embedding = session.run(embedded_text, feed_dict={text_input: [sample_query]})[0]

search_results = vector_index_chunk.get_nns_by_vector(query_embedding, n=num_results)

print('sentences similar to: "{}"\n'.format(sample_query))
# search_results

for idx, result_idx in enumerate(search_results):
  page_num = docs[result_idx]
  text = doc_texts[(doc_texts["_id"] == page_num)]["text"].iloc[0]
  # sentence_results[idx] = (page_num, text)
  print(f"{page_num}: {text}")


sentences similar to: "Subway times are getting worse"

p1384: schedule: East Side Access - A project primarilyof benefit to Long Island Railroad Commuters,ESA is 150% over budget and 14 years late.
p1396: Second Avenue Subway: New Yorkers have been promised a 2nd Ave subway since 1919.
p1405: Metro North: The capital plan calls for $2.5 billion for Metro North projects to benefit Westchester, northern suburb and Connecticut commuters.
p1381: MTA bureaucrats than for service improvements.
p1392: budget. MTA Financial Plan, page I-3. So where do all those increases go?
p1729: We need to focus on getting people between their jobs and their homes, and also connecting them to the world!
p1379: While Service Deteriorates And MTA Waste Increases The State Needs to Focus on Improving Service, Not Just Squeezing More Money Out of NYC Residents for Declining Service Subway riders pay a significantly larger share of the costs of operating the subway than LIRR and Metro North
p1390: Subject: MTA 

**Copyright 2019 The TensorFlow Hub Authors and Quartz.**

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2019 The TensorFlow Hub Authors and Quartz All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================